In [1]:
using PyCall
cma = pyimport("cma")

PyObject <module 'cma' from '/home/mkoehler/.local/lib/python3.8/site-packages/cma/__init__.py'>

In [2]:
using Distributed
N = 10
addprocs(N)

10-element Array{Int64,1}:
  2
  3
  4
  5
  6
  7
  8
  9
 10
 11

In [6]:
@everywhere using DrWatson
@everywhere @quickactivate :Catalyst
#@everywhere include(srcdir("Parser.jl"))

In [4]:
function calibrate(cma,N)
    Init = [0.006536800306, 1., 1., 1.] #initial guess
    opts = cma.CMAOptions()
    opts["bounds"] = [[0, 0, 0, 0], [1, 50, 20, 20]] 
    opts["popsize"] = N
    weights = cma.recombination_weights.RecombinationWeights(N)
    opts["CMA_recombination_weights"] = weights
    es = cma.CMAEvolutionStrategy(Init, 0.5, opts) 
    
    input_exp = []
    output_exp = []
    for row in CSV.File(datadir("experiment/new-data.csv"); delim = " ")
        push!(input_exp, row.I)
        push!(output_exp, row.O)
    end
    
#    microMesh = Parser.getGrid(projectdir("test/catalyst.msh"))
    
    while isempty(es.stop()) 
        solutions = es.ask() 
        fitness = pmap(function eval(input_args) 
                            try 
                               return Catalyst.solve(input_args[1], input_args[2], 1., 
                                          input_exp, output_exp, progress=false, 
                                          microcomp_type=:nonlinear, 
                                          Q=input_args[3], kₙ=input_args[4], 
                                          calibration=true)
                             catch err #in case we loose newton convergence in a microscopic computation 
                                return 1e10
                             end
                        end
                            ,solutions) 
        es.tell(solutions,fitness) 
        es.logger.add() 
        es.disp() 
    end
end

calibrate (generic function with 1 method)

In [ ]:
@time calibrate(cma, N)

(5_w,10)-aCMA-ES (mu_w=3.2,w_1=45%) in dimension 4 (seed=887702, Wed Jul 15 12:44:46 2020)
(5_w,10)-aCMA-ES (mu_w=3.2,w_1=45%) in dimension 4 (seed=921962, Wed Jul 15 12:45:16 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     10 1.756981385054186e+02 1.0e+00 5.41e-01  5e-01  7e-01 16:39.5
    2     20 1.665959450482710e+02 1.5e+00 6.23e-01  5e-01  8e-01 32:31.5
    3     30 1.505769499928413e+02 1.7e+00 7.69e-01  6e-01  1e+00 49:01.4
    4     40 1.171577622314067e+02 2.1e+00 9.83e-01  8e-01  2e+00 66:24.9
    5     50 1.140198538930298e+02 2.5e+00 1.04e+00  8e-01  2e+00 84:31.1
    6     60 1.290544130820885e+02 2.1e+00 9.07e-01  7e-01  1e+00 102:35.2
    7     70 1.168821184065500e+02 2.1e+00 8.92e-01  6e-01  1e+00 121:26.6
    8     80 9.106513294321772e+01 2.2e+00 8.72e-01  6e-01  1e+00 139:54.8
    9     90 8.905030212783787e+01 2.4e+00 7.88e-01  5e-01  9e-01 158:33.6
   10    100 5.439316169528274e+01 2.2e+00 7.52e-01  5e-01  9e-01 177:10.8
 